<a href="https://colab.research.google.com/github/ahan02/NLP_Random_Forest/blob/main/NLP_Disaster_Tweets_%7C_NLP_%7C_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [166]:
!pip install datasets

In [167]:
#Import libs
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from wordcloud import WordCloud
from imblearn.over_sampling import RandomOverSampler
from datasets import Dataset, load_metric
# import random
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, AdamW, get_linear_schedule_with_warmup, DataCollatorWithPadding
# from sklearn.metrics import accuracy_score, precision_recall_fscore_support
# from kaggle_secrets import UserSecretsClient
# import wandb

In [168]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [169]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/datasets/train.csv')
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [170]:
import pandas as pd
df2 = pd.read_csv('/content/drive/MyDrive/datasets/test.csv')
df2.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


Let's drop some null text and cleansing text

In [171]:
df = df.dropna(subset=['text'])

Then remove some punctuation

In [172]:
import string
import re
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [173]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

df['text_cleaned'] = df['text'].apply(lambda x: remove_punct(x))
df2['text_cleaned'] = df2['text'].apply(lambda x: remove_punct(x))


In [174]:
df.head(2)

,id,keyword,location,text,target,text_cleaned
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this earthquake Ma...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask Canada


cleaning test data set too

In [175]:
df2.head(2)

,id,keyword,location,text,text_cleaned
0,0,NaN,NaN,Just happened a terrible car crash,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",Heard about earthquake is different cities sta...


Then we will further cleanse the data by removing stopwords

In [176]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


subset the data with target=1, to identify most frequently used words with target=1.0

In [177]:
# Function to preprocess and tokenize text
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub('[0-9]+', '', text)
    # Tokenize the text
    words = text.split()
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    return words
# Subset only target=1
df_sub=df[df['target']==1]
# Apply the preprocessing function to the text column
df_sub['tokens'] = df_sub['text'].apply(preprocess_text)

# Display the DataFrame with tokens
df_sub['tokens']

<ipython-input-177-4ffe54a1aead>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['tokens'] = df_sub['text'].apply(preprocess_text)


,tokens
0,"[deeds, reason, earthquake, may, allah, forgiv..."
1,"[forest, fire, near, la, ronge, sask, canada]"
2,"[residents, asked, shelter, place, notified, o..."
3,"[people, receive, wildfires, evacuation, order..."
4,"[got, sent, photo, ruby, alaska, smoke, wildfi..."
...,...
7608,"[two, giant, cranes, holding, bridge, collapse..."
7609,"[aria_ahrary, thetawniest, control, wild, fire..."
7610,"[utckm, volcano, hawaii, httptcozdtoydebj]"
7611,"[police, investigating, ebike, collided, car, ..."


In [178]:
import pandas as pd
import re
from collections import Counter
from nltk.corpus import stopwords
import nltk

Let's Try to find most frequently used words

In [179]:
# Combine all tokens into one list
all_tokens = [word for tokens in df_sub['tokens'] for word in tokens]

# Count the frequency of each word
word_counts = Counter(all_tokens)

# Display the most common words
print(word_counts.most_common(1000))
# Get the most common words list
word_list = [word for word, count in word_counts.most_common(1000)]


[('fire', 178), ('news', 140), ('via', 121), ('disaster', 117), ('california', 111), ('suicide', 110), ('police', 107), ('amp', 107), ('people', 105), ('killed', 93), ('like', 92), ('pm', 89), ('hiroshima', 89), ('storm', 85), ('us', 84), ('crash', 84), ('fires', 84), ('families', 81), ('train', 79), ('emergency', 76), ('buildings', 75), ('bomb', 74), ('mh', 72), ('two', 71), ('nuclear', 70), ('attack', 69), ('video', 69), ('wildfire', 69), ('bombing', 67), ('get', 66), ('accident', 66), ('one', 65), ('northern', 64), ('burning', 64), ('dead', 63), ('legionnaires', 62), ('bomber', 60), ('car', 58), ('homes', 58), ('still', 57), ('war', 57), ('im', 56), ('new', 56), ('atomic', 56), ('years', 54), ('obama', 54), ('fatal', 54), ('collapse', 51), ('may', 50), ('forest', 50), ('debris', 50), ('dont', 50), ('watch', 50), ('first', 49), ('japan', 49), ('malaysia', 49), ('man', 48), ('latest', 48), ('mass', 48), ('near', 47), ('rt', 47), ('water', 47), ('severe', 47), ('today', 46), ('earthqua

Create matches column based on frequently used words list

In [180]:
# Define a function to find matching words
def find_matches(tokens):
    return [word for word in tokens if word in word_list]

# Apply the function to each row and create a new 'matches' column
df_sub['matches'] = df_sub['tokens'].apply(find_matches)

# Display the DataFrame with the new 'matches' column
df_sub.head(10)

<ipython-input-180-f92a3d6cdcf3>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['matches'] = df_sub['tokens'].apply(find_matches)


,id,keyword,location,text,target,text_cleaned,tokens,matches
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this earthquake Ma...,"[deeds, reason, earthquake, may, allah, forgiv...","[reason, earthquake, may, allah, us]"
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask Canada,"[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, canada]"
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,All residents asked to shelter in place are be...,"[residents, asked, shelter, place, notified, o...","[residents, shelter, place, evacuation, shelte..."
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation orders in...,"[people, receive, wildfires, evacuation, order...","[people, wildfires, evacuation, orders, califo..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent this photo from Ruby Alaska as s...,"[got, sent, photo, ruby, alaska, smoke, wildfi...","[got, photo, smoke, wildfires, school]"
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1,RockyFire Update California Hwy closed in bo...,"[rockyfire, update, california, hwy, closed, d...","[update, california, hwy, closed, due, lake, c..."
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1,flood disaster Heavy rain causes flash floodin...,"[flood, disaster, heavy, rain, causes, flash, ...","[flood, disaster, heavy, rain, causes, flash, ..."
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1,Im on top of the hill and I can see a fire in ...,"[im, top, hill, see, fire, woods]","[im, top, see, fire]"
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1,Theres an emergency evacuation happening now i...,"[theres, emergency, evacuation, happening, bui...","[theres, emergency, evacuation, happening, bui..."
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1,Im afraid that the tornado is coming to our area,"[im, afraid, tornado, coming, area]","[im, tornado, coming, area]"


 Get the most common 500 words & create narrow search column

In [181]:
# Combine all tokens into one list
all_tokens = [word for tokens in df_sub['matches'] for word in tokens]

# Count the frequency of each word
word_counts = Counter(all_tokens)

# Display the most common words
print(word_counts.most_common(500))
# Get the most common words list
word_list = [word for word, count in word_counts.most_common(500)]


[('fire', 178), ('news', 140), ('via', 121), ('disaster', 117), ('california', 111), ('suicide', 110), ('police', 107), ('amp', 107), ('people', 105), ('killed', 93), ('like', 92), ('pm', 89), ('hiroshima', 89), ('storm', 85), ('us', 84), ('crash', 84), ('fires', 84), ('families', 81), ('train', 79), ('emergency', 76), ('buildings', 75), ('bomb', 74), ('mh', 72), ('two', 71), ('nuclear', 70), ('attack', 69), ('video', 69), ('wildfire', 69), ('bombing', 67), ('get', 66), ('accident', 66), ('one', 65), ('northern', 64), ('burning', 64), ('dead', 63), ('legionnaires', 62), ('bomber', 60), ('car', 58), ('homes', 58), ('still', 57), ('war', 57), ('im', 56), ('new', 56), ('atomic', 56), ('years', 54), ('obama', 54), ('fatal', 54), ('collapse', 51), ('may', 50), ('forest', 50), ('debris', 50), ('dont', 50), ('watch', 50), ('first', 49), ('japan', 49), ('malaysia', 49), ('man', 48), ('latest', 48), ('mass', 48), ('near', 47), ('rt', 47), ('water', 47), ('severe', 47), ('today', 46), ('earthqua

In [182]:
# Define a function to find matching words
def find_matches(tokens):
    return [word for word in tokens if word in word_list]

# Apply the function to each row and create a new 'matches' column
df_sub['narrow_matches'] = df_sub['matches'].apply(find_matches)

# Display the DataFrame with the new 'matches' column
df_sub.head(10)

<ipython-input-182-a952e70bd5ff>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['narrow_matches'] = df_sub['matches'].apply(find_matches)


,id,keyword,location,text,target,text_cleaned,tokens,matches,narrow_matches
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this earthquake Ma...,"[deeds, reason, earthquake, may, allah, forgiv...","[reason, earthquake, may, allah, us]","[earthquake, may, us]"
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask Canada,"[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, canada]","[forest, fire, near, la]"
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,All residents asked to shelter in place are be...,"[residents, asked, shelter, place, notified, o...","[residents, shelter, place, evacuation, shelte...","[place, evacuation, place]"
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation orders in...,"[people, receive, wildfires, evacuation, order...","[people, wildfires, evacuation, orders, califo...","[people, evacuation, california]"
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent this photo from Ruby Alaska as s...,"[got, sent, photo, ruby, alaska, smoke, wildfi...","[got, photo, smoke, wildfires, school]","[got, photo, school]"
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1,RockyFire Update California Hwy closed in bo...,"[rockyfire, update, california, hwy, closed, d...","[update, california, hwy, closed, due, lake, c...","[update, california, closed, due, county, fire]"
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1,flood disaster Heavy rain causes flash floodin...,"[flood, disaster, heavy, rain, causes, flash, ...","[flood, disaster, heavy, rain, causes, flash, ...","[flood, disaster, heavy, rain, flash, flooding]"
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1,Im on top of the hill and I can see a fire in ...,"[im, top, hill, see, fire, woods]","[im, top, see, fire]","[im, top, see, fire]"
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1,Theres an emergency evacuation happening now i...,"[theres, emergency, evacuation, happening, bui...","[theres, emergency, evacuation, happening, bui...","[theres, emergency, evacuation, building, acro..."
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1,Im afraid that the tornado is coming to our area,"[im, afraid, tornado, coming, area]","[im, tornado, coming, area]","[im, tornado, coming, area]"


In [183]:
!pip install python-Levenshtein

The Levenshtein distance (LV distance) is a metric for measuring the difference between two sequences. Specifically, it quantifies how dissimilar two strings are by counting the minimum number of single-character edits (insertions, deletions, or substitutions) required to transform one string into the other.

Try to find words list that has greater than 0.90 of LV ratio, create column on match using the most frequent 500 words list

In [184]:
import Levenshtein

# Define a function to find matches based on Levenshtein ratio
def find_levenshtein_matches(tokens, min_ratio=0.9):
    matches = []
    for token in tokens:
        for word in word_list:
            if Levenshtein.ratio(token.lower(), word.lower()) >= min_ratio:
                matches.append(word)
    return matches

# Assuming df_sub has a column named 'matches' containing the tokens to search for
# Convert 'matches' column to list of tokens if necessary
df_sub['matches'] = df_sub['matches'].apply(lambda x: x.split() if isinstance(x, str) else x)

# Apply the function to each row and create a new 'narrow_matches' column
df_sub['narrow_lv_matches'] = df_sub['matches'].apply(find_levenshtein_matches)

# Display the DataFrame with the new 'narrow_matches' column
df_sub.head(10)

<ipython-input-184-4d7b0c418d2f>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['matches'] = df_sub['matches'].apply(lambda x: x.split() if isinstance(x, str) else x)
<ipython-input-184-4d7b0c418d2f>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['narrow_lv_matches'] = df_sub['matches'].apply(find_levenshtein_matches)


,id,keyword,location,text,target,text_cleaned,tokens,matches,narrow_matches,narrow_lv_matches
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this earthquake Ma...,"[deeds, reason, earthquake, may, allah, forgiv...","[reason, earthquake, may, allah, us]","[earthquake, may, us]","[earthquake, may, us]"
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask Canada,"[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, canada]","[forest, fire, near, la]","[forest, fire, near, la]"
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,All residents asked to shelter in place are be...,"[residents, asked, shelter, place, notified, o...","[residents, shelter, place, evacuation, shelte...","[place, evacuation, place]","[place, evacuation, place]"
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation orders in...,"[people, receive, wildfires, evacuation, order...","[people, wildfires, evacuation, orders, califo...","[people, evacuation, california]","[people, wildfire, evacuation, california]"
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent this photo from Ruby Alaska as s...,"[got, sent, photo, ruby, alaska, smoke, wildfi...","[got, photo, smoke, wildfires, school]","[got, photo, school]","[got, photo, wildfire, school]"
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1,RockyFire Update California Hwy closed in bo...,"[rockyfire, update, california, hwy, closed, d...","[update, california, hwy, closed, due, lake, c...","[update, california, closed, due, county, fire]","[update, california, closed, due, county, fire..."
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1,flood disaster Heavy rain causes flash floodin...,"[flood, disaster, heavy, rain, causes, flash, ...","[flood, disaster, heavy, rain, causes, flash, ...","[flood, disaster, heavy, rain, flash, flooding]","[floods, flood, disaster, heavy, rain, cause, ..."
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1,Im on top of the hill and I can see a fire in ...,"[im, top, hill, see, fire, woods]","[im, top, see, fire]","[im, top, see, fire]","[im, top, see, fire]"
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1,Theres an emergency evacuation happening now i...,"[theres, emergency, evacuation, happening, bui...","[theres, emergency, evacuation, happening, bui...","[theres, emergency, evacuation, building, acro...","[theres, emergency, evacuation, buildings, bui..."
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1,Im afraid that the tornado is coming to our area,"[im, afraid, tornado, coming, area]","[im, tornado, coming, area]","[im, tornado, coming, area]","[im, tornado, coming, area]"


The Longest Common Substring (LCS) problem is about finding the longest substring that is common to two given strings. Unlike the Longest Common Subsequence problem, which deals with subsequences (where characters don't need to be contiguous), the Longest Common Substring requires the characters to be contiguous.

find words in word_list that have a Longest Common Substring (LCS) similarity with each token in a list of tokens. Here's a detailed explanation of how it works:

minimum proportion of the token length that should match the longest common substring to be considered a match is 90%


In [185]:
def lcs(X, Y):
    m = len(X)
    n = len(Y)
    L = [[0] * (n + 1) for _ in range(m + 1)]

    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0 or j == 0:
                L[i][j] = 0
            elif X[i - 1] == Y[j - 1]:
                L[i][j] = L[i - 1][j - 1] + 1
            else:
                L[i][j] = max(L[i - 1][j], L[i][j - 1])

    return L[m][n]

def find_lcs_matches(tokens, threshold=0.9):
    matches = []
    for token in tokens:
        for word in word_list:
            lcs_length = lcs(token.lower(), word.lower())
            if lcs_length / len(token) >= threshold:
                matches.append(word)
    return matches

# Apply the function to each row and create a new 'matches' column
df_sub['narrow_partial_lcs_matches'] = df_sub['matches'].apply(find_matches)

# Display the DataFrame with the new 'matches' column
df_sub.head(10)

<ipython-input-185-cda2fef5c72f>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['narrow_partial_lcs_matches'] = df_sub['matches'].apply(find_matches)


,id,keyword,location,text,target,text_cleaned,tokens,matches,narrow_matches,narrow_lv_matches,narrow_partial_lcs_matches
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this earthquake Ma...,"[deeds, reason, earthquake, may, allah, forgiv...","[reason, earthquake, may, allah, us]","[earthquake, may, us]","[earthquake, may, us]","[earthquake, may, us]"
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask Canada,"[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, canada]","[forest, fire, near, la]","[forest, fire, near, la]","[forest, fire, near, la]"
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,All residents asked to shelter in place are be...,"[residents, asked, shelter, place, notified, o...","[residents, shelter, place, evacuation, shelte...","[place, evacuation, place]","[place, evacuation, place]","[place, evacuation, place]"
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation orders in...,"[people, receive, wildfires, evacuation, order...","[people, wildfires, evacuation, orders, califo...","[people, evacuation, california]","[people, wildfire, evacuation, california]","[people, evacuation, california]"
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent this photo from Ruby Alaska as s...,"[got, sent, photo, ruby, alaska, smoke, wildfi...","[got, photo, smoke, wildfires, school]","[got, photo, school]","[got, photo, wildfire, school]","[got, photo, school]"
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1,RockyFire Update California Hwy closed in bo...,"[rockyfire, update, california, hwy, closed, d...","[update, california, hwy, closed, due, lake, c...","[update, california, closed, due, county, fire]","[update, california, closed, due, county, fire...","[update, california, closed, due, county, fire]"
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1,flood disaster Heavy rain causes flash floodin...,"[flood, disaster, heavy, rain, causes, flash, ...","[flood, disaster, heavy, rain, causes, flash, ...","[flood, disaster, heavy, rain, flash, flooding]","[floods, flood, disaster, heavy, rain, cause, ...","[flood, disaster, heavy, rain, flash, flooding]"
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1,Im on top of the hill and I can see a fire in ...,"[im, top, hill, see, fire, woods]","[im, top, see, fire]","[im, top, see, fire]","[im, top, see, fire]","[im, top, see, fire]"
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1,Theres an emergency evacuation happening now i...,"[theres, emergency, evacuation, happening, bui...","[theres, emergency, evacuation, happening, bui...","[theres, emergency, evacuation, building, acro...","[theres, emergency, evacuation, buildings, bui...","[theres, emergency, evacuation, building, acro..."
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1,Im afraid that the tornado is coming to our area,"[im, afraid, tornado, coming, area]","[im, tornado, coming, area]","[im, tornado, coming, area]","[im, tornado, coming, area]","[im, tornado, coming, area]"


In [186]:
df.head(2)

,id,keyword,location,text,target,text_cleaned
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this earthquake Ma...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask Canada


join the subset (target=1.0) with original train data set
and create column frequent_words that contains LV match result. From initial review, this match column looks better than LCS match result (finding more diaster related word list e.g. wildfires and wildfire)

In [187]:
join_df = pd.merge(df, df_sub, on='id', how='left', suffixes=('', '_df2'))
# Drop all columns with the '_df2' suffix
columns_to_drop = join_df.filter(like='_df2').columns
join_df = join_df.drop(columns=columns_to_drop)


In [188]:
def combine_tokens(tokens):
    if isinstance(tokens, list):
        return ' '.join(tokens)
    else:
        # If not a list, convert to a list (or handle appropriately)
        return str(tokens)

# Apply the function to the 'tokens' column and create a new column with the combined sentence
join_df['frequent_words'] = join_df['narrow_lv_matches'].apply(combine_tokens)

In [189]:
join_df.head(2)

,id,keyword,location,text,target,text_cleaned,tokens,matches,narrow_matches,narrow_lv_matches,narrow_partial_lcs_matches,frequent_words
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this earthquake Ma...,"[deeds, reason, earthquake, may, allah, forgiv...","[reason, earthquake, may, allah, us]","[earthquake, may, us]","[earthquake, may, us]","[earthquake, may, us]",earthquake may us
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask Canada,"[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, canada]","[forest, fire, near, la]","[forest, fire, near, la]","[forest, fire, near, la]",forest fire near la


fill '' or null values for target=0
fill '' or null values that didn't return result from LV

In [190]:
# Fill the 'text2' column with values from 'text' column where 'target' is 0
join_df.loc[join_df['target'] == 0, 'frequent_words'] = join_df['text_cleaned']


In [191]:
# Fill remaining null values in 'text2' with values from 'text'
join_df['frequent_words'] = join_df['frequent_words'].fillna(join_df['text_cleaned'])
# Replace empty strings in 'frequent_words' with values from 'text'
join_df['frequent_words'] = join_df.apply(
    lambda row: row['text_cleaned'] if row['frequent_words'] == '' else row['frequent_words'],
    axis=1
)

In [192]:
# Display all columns of the DataFrame
pd.set_option('display.max_columns', None)


Create sparse matrix using X as frequently used words from tweets with target=1.0

And fit this with y (target column from train data set)

In [193]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


In [194]:
# Step 2: Extract TF-IDF features
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(join_df['frequent_words'])
# Convert the sparse matrix to a dense matrix
X_dense = X.todense()

# Create a DataFrame from the dense matrix
feature_names = vectorizer.get_feature_names_out()
X_df = pd.DataFrame(X_dense, columns=feature_names)

# Display the DataFrame
X_df.head(5)

y = join_df['target']

# Step 3: Train a Random Forest classifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

# Step 4: Make predictions and evaluate the model
y_pred = classifier.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.94      0.96       874
           1       0.92      0.98      0.95       649

    accuracy                           0.95      1523
   macro avg       0.95      0.96      0.95      1523
weighted avg       0.96      0.95      0.95      1523




**Precision:**

Precision measures the accuracy of positive predictions. It is the ratio of true positives (correctly predicted positive instances) to the sum of true positives and false positives (all instances classified as positive).

Formula:
Precision = True Positives / (True Positives + False Positives)


**Recall:**

Recall (or Sensitivity) measures the ability of the model to identify all relevant instances of the positive class. It is the ratio of true positives to the sum of true positives and false negatives (all actual positive instances).
Formula:
Recall = True Positives / (True Positives + False Negatives)

**F1-Score:**

The F1-score is the harmonic mean of precision and recall. It balances the two metrics and is useful when you need a single metric to evaluate the performance.
Formula:
F1-Score = 2 * (Precision * Recall) / (Precision + Recall)



Class 1
Precision: 0.92
Out of all instances predicted as Class 1, 92% are actually Class 1.

Recall: 0.98
Out of all actual Class 1 instances, 98% are correctly identified.

F1-Score: 0.95
The harmonic mean of precision and recall for Class 1.


**Accuracy:**

Accuracy: 0.95
The proportion of correctly predicted instances (both Class 0 and Class 1) out of all instances.
Formula: (True Positives + True Negatives) / Total Instances
Here, 95% of the total 1523 instances are correctly classified.


The Random Forest classifier is a popular and versatile machine learning algorithm used for classification and regression tasks. It operates as an ensemble learning method that combines multiple decision trees to improve model performance and robustness.

***Creates a Random Forest classifier with 100 decision trees for prediction***



Random Forest is an ensemble learning method, which means it builds multiple models and combines their predictions to produce a final result.

**Decision Trees:**

A decision tree is a model that makes decisions based on a series of questions or splits. Each node represents a decision point, and each branch represents the outcome of a decision.

**Bootstrap Aggregation (Bagging):**

It creates multiple subsets of the training data by random sampling with replacement. Each subset is used to train a different decision tree. This helps to reduce overfitting.

**Random Feature Selection:**

For each decision tree, Random Forest selects a random subset of features at each split node rather than using all features. This
helps in reducing correlation between them.

**Building Trees:**

Each decision tree in the forest is built using one of the bootstrapped subsets and the randomly selected features.

**Aggregation of Predictions:**

The final prediction is determined by taking a majority vote among the predictions of all individual trees.


Then use df2 (test.csv) to do prediction

In [195]:
# Example validation DataFrame

# Transform the validation data using the same TF-IDF vectorizer
X_validation = vectorizer.transform(df2['text_cleaned'])
X_validation_dense = X_validation.todense()
X_validation_df = pd.DataFrame(X_validation_dense, columns=feature_names)

# Make predictions on the validation set
y_validation_pred = classifier.predict(X_validation_df)

# If you have no target labels for validation, you can simply display predictions
print("Predictions for validation set:")
# Create a DataFrame with the validation ids and predictions
validation_results = pd.DataFrame({
    'id': df2['id'],
    'target': y_validation_pred
})
validation_results

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


Predictions for validation set:


,id,target
0,0,1
1,2,0
2,3,0
3,9,1
4,11,0
...,...,...
3258,10861,1
3259,10865,0
3260,10868,0
3261,10874,1


In [196]:
validation_results.to_csv('/content/drive/MyDrive/datasets/sample_submission.csv', index=False)